In [2]:
import pandas as pd
import numpy as np

In [5]:
train_data = pd.read_csv("volume_entry_train_1S.csv")
train_data.columns

Index([u'Unnamed: 0', u'cargo_count0', u'cargo_count1', u'cargo_count2',
       u'cargo_count3', u'cargo_count4', u'cargo_count5', u'cargo_model0',
       u'cargo_model1', u'cargo_model2', u'cargo_model3', u'cargo_model4',
       u'cargo_model5', u'cargo_model_avg0', u'cargo_model_avg1',
       u'cargo_model_avg2', u'cargo_model_avg3', u'cargo_model_avg4',
       u'cargo_model_avg5', u'etc_count0', u'etc_count1', u'etc_count2',
       u'etc_count3', u'etc_count4', u'etc_count5', u'no_count0', u'no_count1',
       u'no_count2', u'no_count3', u'no_count4', u'no_count5',
       u'passenger_count0', u'passenger_count1', u'passenger_count2',
       u'passenger_count3', u'passenger_count4', u'passenger_count5',
       u'passenger_model0', u'passenger_model1', u'passenger_model2',
       u'passenger_model3', u'passenger_model4', u'passenger_model5',
       u'passenger_model_avg0', u'passenger_model_avg1',
       u'passenger_model_avg2', u'passenger_model_avg3',
       u'passenger_model_avg4',

In [28]:
# 训练集做的不对，六个时段模型的训练集应该是分开的，但是这里合在一块了，区别就是下面这几个字段
del train_data["month"]
del train_data["day"]
del train_data["hour"]
del train_data["minute"]
train_data["time"] = train_data["Unnamed: 0"]
del train_data["Unnamed: 0"]

In [30]:
# 获取每一天6点到8点的流量，和8点到10点的流量
train_data["time"] = train_data["time"].apply(lambda x: pd.Timestamp(x))
train_data["month"] = train_data["time"].apply(lambda x: x.month)
train_data["day"] = train_data["time"].apply(lambda x: x.day)
train_data["hour"] = train_data["time"].apply(lambda x: x.hour)
train_data["minute"] = train_data["time"].apply(lambda x: x.minute)
train_6_8 = train_data[(train_data["hour"] >= 6) & (train_data["hour"] < 8)][["y", "day", "month"]]
train_6_8.index = train_data[(train_data["hour"] >= 6) & (train_data["hour"] < 8)]["time"]
train_8_10 = train_data[(train_data["hour"] >= 8) & (train_data["hour"] < 10)][["y", "day", "month"]]
train_8_10.index = train_data[(train_data["hour"] >= 8) & (train_data["hour"] < 10)]["time"]

In [41]:
# 按天拆分成无数个元素
train_time_lst = [train_6_8[(train_6_8["day"] == i) & (train_6_8["month"] == 9)]["y"] for i in range(19, 31, 1)]
train_time_lst += [train_6_8[(train_6_8["day"] == i) & (train_6_8["month"] == 10)]["y"] for i in range(1, 18, 1)]
predict_time_lst = [train_8_10[(train_8_10["day"] == i) & (train_8_10["month"] == 9)]["y"] for i in range(19, 31, 1)]
predict_time_lst += [train_8_10[(train_8_10["day"] == i) & (train_8_10["month"] == 10)]["y"] for i in range(1, 18, 1)]

In [42]:
predict_time_lst

[time
 2016-09-19 08:00:00    44.0
 2016-09-19 08:20:00    44.0
 2016-09-19 08:40:00    44.0
 2016-09-19 09:00:00    39.0
 2016-09-19 09:20:00    30.0
 2016-09-19 09:40:00    34.0
 Name: y, dtype: float64, time
 2016-09-20 08:00:00    56.0
 2016-09-20 08:20:00    60.0
 2016-09-20 08:40:00    63.0
 2016-09-20 09:00:00    57.0
 2016-09-20 09:20:00    64.0
 2016-09-20 09:40:00    50.0
 Name: y, dtype: float64, time
 2016-09-21 08:00:00    36.0
 2016-09-21 08:20:00    57.0
 2016-09-21 08:40:00    37.0
 2016-09-21 09:00:00    32.0
 2016-09-21 09:20:00    27.0
 2016-09-21 09:40:00    32.0
 Name: y, dtype: float64, time
 2016-09-22 08:00:00    47.0
 2016-09-22 08:20:00    49.0
 2016-09-22 08:40:00    43.0
 2016-09-22 09:00:00    27.0
 2016-09-22 09:20:00    35.0
 2016-09-22 09:40:00    49.0
 Name: y, dtype: float64, time
 2016-09-23 08:00:00    58.0
 2016-09-23 08:20:00    53.0
 2016-09-23 08:40:00    56.0
 2016-09-23 09:00:00    56.0
 2016-09-23 09:20:00    45.0
 2016-09-23 09:40:00    41.0


In [43]:
test_list = pd.Series([1,2,3,4,5,6])
def score(list1, list2):
    temp1 = pd.Series(list1.values)
    temp2 = pd.Series(list2.values)
    return ((temp1 - temp2) ** 2).mean()
score_list = pd.Series([score(test_list, train) for train in train_time_lst], index = range(29))
predict_result = predict_time_lst[score_list.idxmin()]
predict_result

time
2016-10-08 08:00:00    37.0
2016-10-08 08:20:00    41.0
2016-10-08 08:40:00    34.0
2016-10-08 09:00:00    25.0
2016-10-08 09:20:00    28.0
2016-10-08 09:40:00    24.0
Name: y, dtype: float64